<a href="https://colab.research.google.com/github/s0uravk/Practice/blob/main/Geo_api_airports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
# Import API key
geoapify_key = "2ed98981ff9b4fbb80a82925225c922d"

In [123]:
data = {}
data = pd.read_csv("Cities.csv")
data

,City
0,Sydney
1,Melbourne
2,Perth
3,Cairns
4,Adelaide
5,Gold Coast
6,Townsville
7,Williamtown


In [124]:
data["Lat"] = ""
data["Lon"] = ""
data["Airport Name"] = ""
data["IATA Name"] = ""
data["Airport Address"] = ""
data["Distance"] = ""
data["Website"] = ""
data.head()

,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,,,,,,,
1,Melbourne,,,,,,,
2,Perth,,,,,,,
3,Cairns,,,,,,,
4,Adelaide,,,,,,,


In [125]:
 # Define the API parameters
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

In [126]:
# Print a message to follow up the airport search
print("Starting airport search")

# Loop through the data DataFrame and search coordinates for each city
for index, row in data.iterrows():

    # Get the city's name & add ", Australia" to the string so geoapify finds the correct city
    city = f'{row["City"]}, Australia'

    # Add the current city to the parameters
    params["text"] = city

    # Make the API request
    response = requests.get(base_url, params = params)

    # Convert response to JSON
    output = response.json()
    #pprint(output)

    #Extract latitude and longitude
    lat = output["results"][0]["lat"]
    lon = output["results"][0]["lon"]

    # Log the search results

    data.loc[index, "Lat"] = lat
    data.loc[index, "Lon"] = lon

    print(f"Coordinates for {city} fetched...")

# Display sample data to confirm that the coordinates appear
print(data)

Starting airport search
Coordinates for Sydney, Australia fetched...
Coordinates for Melbourne, Australia fetched...
Coordinates for Perth, Australia fetched...
Coordinates for Cairns, Australia fetched...
Coordinates for Adelaide, Australia fetched...
Coordinates for Gold Coast, Australia fetched...
Coordinates for Townsville, Australia fetched...
Coordinates for Williamtown, Australia fetched...
          City        Lat         Lon Airport Name IATA Name Airport Address  \
0       Sydney -33.869844  151.208285                                          
1    Melbourne -37.814245  144.963173                                          
2        Perth -31.955893  115.860585                                          
3       Cairns -16.920666  145.772185                                          
4     Adelaide -34.928181  138.599931                                          
5   Gold Coast -28.002373  153.414599                                          
6   Townsville -19.256939  146.823954  

In [127]:
# Set parameters to search for airport's details
radius = 50000
params = {
    "categories":"airport",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the airport search
print("Starting airport details search")

# Iterate through the types_df DataFrame
for index, row in data.iterrows():

    # Get the city's name
    city = row["City"]

    # Get latitude, longitude from the DataFrame
    lon = row["Lon"]
    lat = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary  # Convert the API response to JSON format
    response = requests.get(base_url, params = params).json()
    # pprint(response)
    # break


    # Grab the first airport from the results and store the details in the DataFrame
    try:
     data.loc[index, "Airport Name"] =  response["features"][0]["properties"]["address_line1"]
     data.loc[index, "Airport Address"] = response["features"][0]["properties"]["address_line2"]
     data.loc[index, "IATA Name"] = response["features"][0]["properties"]["datasource"]["raw"]["iata"]
     data.loc[index, "Distance"] = response["features"][0]["properties"]["distance"]
     data.loc[index, "Website"] = response["features"][0]["properties"]["datasource"]["raw"]["website"]
    except KeyError as e:
        # If no airport is found, log the error.
        print(f"{e.args[0]} not found for {data.loc[index, 'Airport Name']}")

    # Log the search results
    print(f"nearest airport from {city}: {data.loc[index, 'Airport Name']}")

#Display sample data
print(data)

Starting airport details search
nearest airport from Sydney: Sydney (Kingsford Smith) Airport
nearest airport from Melbourne: Essendon Fields Airport
nearest airport from Perth: Horrie Miller Drive
website not found for Cairns International Airport
nearest airport from Cairns: Cairns International Airport
nearest airport from Adelaide: Anna Meares Cycleway
nearest airport from Gold Coast: Gold Coast Airport
nearest airport from Townsville: Townsville Airport
nearest airport from Williamtown: Newcastle Airport
          City        Lat         Lon                      Airport Name  \
0       Sydney -33.869844  151.208285  Sydney (Kingsford Smith) Airport   
1    Melbourne -37.814245  144.963173           Essendon Fields Airport   
2        Perth -31.955893  115.860585               Horrie Miller Drive   
3       Cairns -16.920666  145.772185      Cairns International Airport   
4     Adelaide -34.928181  138.599931              Anna Meares Cycleway   
5   Gold Coast -28.002373  153.4145